In [109]:
import os
os.getcwd()
os.chdir('C:/Users/mgabr/Desktop/HLT/project/BioASQ2020')

In [116]:
${fileDirname}

SyntaxError: invalid syntax (<ipython-input-116-92d7e61cd2cd>, line 1)

# List question

Il modello per le liste si basa sullo stesso modello utilizzato per le factoid provvediamo a importare il modello corretto e facciamo un test sul funzionamento

In [1]:
from transformers import BertTokenizer, BertModel, BertForQuestionAnswering
import torch
tokenizer = BertTokenizer.from_pretrained("./transformers_models/biobert_factoid_pytorch")
model = BertForQuestionAnswering.from_pretrained("./transformers_models/biobert_factoid_pytorch")

OSError: Model name './transformers_models/biobert_factoid_pytorch' was not found in tokenizers model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). We assumed './transformers_models/biobert_factoid_pytorch' was a path, a model identifier, or url to a directory containing vocabulary files named ['vocab.txt'] but couldn't find such vocabulary files at this path or url.

La risposta corretta contiene queste entità

In [111]:
from utils.data import load_data_list, clean_synonyms
lista_domande=load_data_list("./data/training8b.json")
lista_domande[0][1]

[['epidermal growth factor'],
 ['betacellulin'],
 ['epiregulin'],
 ['heparin-binding epidermal growth factor'],
 ['transforming growth factor-α'],
 ['amphiregulin'],
 ['epigen']]

Definisco la funzione per stampare la risposta corretta

In [113]:
def print_answer(answer_start,answer_end,all_tokens):# Start with the first token.
    answer = all_tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if all_tokens[i][0:2] == '##':
            answer += all_tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + all_tokens[i]

    print('Answer: "' + answer + '"')

In [114]:
question, text = "List signaling molecules (ligands) that interact with the receptor EGFR?", "the epidermal growth factor receptor (EGFR) ligands, such as epidermal growth factor (EGF) and amphiregulin (AREG)"
encoding = tokenizer.encode_plus(question, text)
input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(all_tokens)
print(input_ids)
print(token_type_ids)

['[CLS]', 'list', 'signaling', 'molecules', '(', 'l', '##igan', '##ds', ')', 'that', 'interact', 'with', 'the', 'receptor', 'e', '##g', '##f', '##r', '?', '[SEP]', 'the', 'e', '##pid', '##er', '##mal', 'growth', 'factor', 'receptor', '(', 'e', '##g', '##f', '##r', ')', 'l', '##igan', '##ds', ',', 'such', 'as', 'e', '##pid', '##er', '##mal', 'growth', 'factor', '(', 'e', '##g', '##f', ')', 'and', 'am', '##phi', '##re', '##gu', '##lin', '(', 'are', '##g', ')', '[SEP]']
[101, 2190, 16085, 10799, 113, 181, 10888, 3680, 114, 1115, 12254, 1114, 1103, 10814, 174, 1403, 2087, 1197, 136, 102, 1103, 174, 25786, 1200, 7435, 3213, 5318, 10814, 113, 174, 1403, 2087, 1197, 114, 181, 10888, 3680, 117, 1216, 1112, 174, 25786, 1200, 7435, 3213, 5318, 113, 174, 1403, 2087, 114, 1105, 1821, 27008, 1874, 13830, 2836, 113, 1132, 1403, 114, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

____________________________
## 1 Approccio

Dato il vettore della softmax per la risposta prendiamo tutte le parole che riportano uno score superiore a un determinato threshold. Prendiamo poi il minimo del vettore dello start e il massimo del vettore dell'end e poi utilizziamo un NER per identificare le entità

In [101]:
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
print(start_scores)
print(end_scores)
values_start, indices_start=torch.topk(start_scores,3)
values_end, indices_end=torch.topk(end_scores,3)
print(indices_start)
print(indices_end)

tensor([[-3.5126, -7.0865, -6.9067, -7.7472, -7.6168, -6.4265, -7.8138, -8.5346,
         -8.5282, -7.4421, -7.2022, -7.1606, -7.4848, -8.1219, -7.4963, -7.8732,
         -8.4675, -8.8533, -7.7126, -7.8307, -2.8615, -3.0179, -6.7289, -6.7677,
         -6.4157, -6.0162, -6.7201, -4.8849, -4.4312, -5.3506, -7.3976, -7.3558,
         -6.2990, -4.9410, -2.0951, -5.2530, -4.9682, -5.6626, -3.1414, -6.0405,
         -0.9388, -7.2085, -7.6842, -6.7369, -5.8507, -6.5148, -5.1844, -4.1726,
         -7.7461, -7.0326, -7.0507, -7.4014, -1.8171, -7.4988, -7.5553, -7.1843,
         -5.8848, -5.8190, -6.7066, -7.4752, -6.2458, -7.8307]],
       grad_fn=<SqueezeBackward1>)
tensor([[-2.4085, -8.4290, -8.2037, -6.7629, -8.2680, -8.3713, -8.0804, -6.4559,
         -7.4831, -8.3483, -8.1218, -8.0534, -8.5519, -7.9593, -8.7100, -8.3001,
         -7.9757, -6.8480, -7.3486, -2.5235, -5.5465, -5.8049, -7.4287, -7.5961,
         -7.3014, -6.4549, -4.0458, -2.8896, -5.6771, -8.3694, -8.4386, -8.0441,
         

Stiamo prendendo l'indice più basso per lo start e quello maggiore per l'end

In [103]:
print_answer(34,56,all_tokens)

Answer: "ligands , such as epidermal growth factor ( egf ) and amphiregulin"


A questo punto si dovrebbe fare NER per identificare le entità

_________________________________
## 2 Approccio

Una volta ottenuto il vettore con gli score si selezionanano le combinazioni di start e end che riportano uno score più alto. Si fa una semplice somma.


In [104]:
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
print(start_scores)
print(end_scores)

tensor([[-3.5126, -7.0865, -6.9067, -7.7472, -7.6168, -6.4265, -7.8138, -8.5346,
         -8.5282, -7.4421, -7.2022, -7.1606, -7.4848, -8.1219, -7.4963, -7.8732,
         -8.4675, -8.8533, -7.7126, -7.8307, -2.8615, -3.0179, -6.7289, -6.7677,
         -6.4157, -6.0162, -6.7201, -4.8849, -4.4312, -5.3506, -7.3976, -7.3558,
         -6.2990, -4.9410, -2.0951, -5.2530, -4.9682, -5.6626, -3.1414, -6.0405,
         -0.9388, -7.2085, -7.6842, -6.7369, -5.8507, -6.5148, -5.1844, -4.1726,
         -7.7461, -7.0326, -7.0507, -7.4014, -1.8171, -7.4988, -7.5553, -7.1843,
         -5.8848, -5.8190, -6.7066, -7.4752, -6.2458, -7.8307]],
       grad_fn=<SqueezeBackward1>)
tensor([[-2.4085, -8.4290, -8.2037, -6.7629, -8.2680, -8.3713, -8.0804, -6.4559,
         -7.4831, -8.3483, -8.1218, -8.0534, -8.5519, -7.9593, -8.7100, -8.3001,
         -7.9757, -6.8480, -7.3486, -2.5235, -5.5465, -5.8049, -7.4287, -7.5961,
         -7.3014, -6.4549, -4.0458, -2.8896, -5.6771, -8.3694, -8.4386, -8.0441,
         

In [105]:
import math  
from queue import PriorityQueue 
def KMaxCombinations( arr1, arr2, dim, K): 
      
    # max heap. 
    pq = PriorityQueue()  
      
    # insert all the possible   
    # combinations in max heap. 
    for i in range(0, dim): 
        for j in range(0, dim): 
           a = arr1[i] + arr2[j]  
           pq.put((-a, a,i,j)) 
              
    # pop first N elements from  
    # max heap and display them. 
    count = 0
    while (count < K): 
        print(pq.get()[0:4]) 
        count = count + 1

In [106]:
KMaxCombinations(start_scores.detach().numpy()[0],end_scores.detach().numpy()[0],len(end_scores.detach().numpy()[0]),5)

(0.89879876, -0.89879876, 40, 36)
(1.4251255, -1.4251255, 40, 56)
(1.7770776, -1.7770776, 52, 36)
(2.0551105, -2.0551105, 34, 36)
(2.3034043, -2.3034043, 52, 56)


Stampiamo per esempio le prime 3 entità escludendo quelle con lo start maggiore dell'end

In [108]:
print_answer(40,56,all_tokens)
print_answer(34,36,all_tokens)
print_answer(52,56,all_tokens)

Answer: "epidermal growth factor ( egf ) and amphiregulin"
Answer: "ligands"
Answer: "amphiregulin"
